In [1]:
# Costco Location Ensemble Techniques

In [2]:
import warnings
warnings.filterwarnings('ignore')

In [3]:
import numpy as np
import pandas as pd
from pathlib import Path
from collections import Counter

In [4]:
from sklearn.metrics import confusion_matrix, accuracy_score, classification_report
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.preprocessing import StandardScaler
#from sklearn.ensemble import AdaBoostClassifier
#from imblearn.ensemble import EasyEnsembleClassifier
from sklearn.metrics import balanced_accuracy_score

In [5]:
import os
# Find the latest version of spark 3.0  from http://www.apache.org/dist/spark/ and enter as the spark version
spark_version = 'spark-3.2.2'
os.environ['SPARK_VERSION']=spark_version

# Install Spark and Java
!apt-get update
!apt-get install openjdk-11-jdk-headless -qq > /dev/null
!wget -q http://www.apache.org/dist/spark/$SPARK_VERSION/$SPARK_VERSION-bin-hadoop2.7.tgz
!tar xf $SPARK_VERSION-bin-hadoop2.7.tgz
!pip install -q findspark

# Set Environment Variables
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-11-openjdk-amd64"
os.environ["SPARK_HOME"] = f"/content/{spark_version}-bin-hadoop2.7"


Ign:1 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Get:2 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease [3,626 B]
Hit:3 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Hit:4 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Hit:5 http://archive.ubuntu.com/ubuntu bionic InRelease
Get:6 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]
Get:7 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease [15.9 kB]
Get:8 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Get:9 http://archive.ubuntu.com/ubuntu bionic-backports InRelease [83.3 kB]
Hit:10 http://ppa.launchpad.net/cran/libgit2/ubuntu bionic InRelease
Hit:11 http://ppa.launchpad.net/deadsnakes/ppa/ubuntu bionic InRelease
Hit:12 http://ppa.launchpad.net/graphics-drivers/ppa/ubuntu bionic InRelease
Get:14 http://ppa.launchpad.net

In [6]:
# Start a SparkSession
import findspark
findspark.init()

In [7]:
!wget https://jdbc.postgresql.org/download/postgresql-42.2.9.jar

--2022-11-03 23:37:18--  https://jdbc.postgresql.org/download/postgresql-42.2.9.jar
Resolving jdbc.postgresql.org (jdbc.postgresql.org)... 72.32.157.228, 2001:4800:3e1:1::228
Connecting to jdbc.postgresql.org (jdbc.postgresql.org)|72.32.157.228|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 914037 (893K) [application/java-archive]
Saving to: ‘postgresql-42.2.9.jar’

postgresql-42.2.9.j 100%[===================>] 892.61K  5.53MB/s    in 0.2s    

2022-11-03 23:37:19 (5.53 MB/s) - ‘postgresql-42.2.9.jar’ saved [914037/914037]



In [8]:
from pyspark.sql import SparkSession
findspark.init()
spark = SparkSession.builder.appName("CloudETL").getOrCreate()

# Read the CSV and Perform Basic Data Cleaning

In [17]:
from pyspark import SparkFiles
url="https://gwufphearingaids.s3.amazonaws.com/ml_data.csv"  #S3 bucket URL
spark.sparkContext.addFile(url)
ml_data_spark_df = spark.read.csv(SparkFiles.get("ml_data.csv"), sep=",", header=True, inferSchema=True)

# Show DataFrame
ml_data_spark_df.show()

+---+----+---------------+-------+-------------------------------------+-------------------------------------+-------------------------------------+-------------------------------------+-------------------------------------+-------------------------------------+-------------------------------------+-------------------------------------+-------------------------------------+-------------------------------------+------------------------------------------------------+------------------------------+---------------------------------------+-------------------------------------+--------------------------------------------+----------------------------------------+----------------------------------+----------------------------------+---------------------------------+--------------------------------------------+----------------------------------+------------------------------------+------------------------------------+------------------------------------+----------------------------------+------

In [18]:
ml_data_df = ml_data_spark_df.toPandas()

In [19]:
ml_data_df.COSTCO_HEARING_CENTER.unique()

array([None, 'Yes', 'No'], dtype=object)

In [20]:
columns = ['ZIP', 
           '2019_Total_Population:_20_to_24_Years', '2019_Total_Population:_25_to_29_Years', 
           '2019_Total_Population:_30_to_34_Years', '2019_Total_Population:_35_to_39_Years', 
           '2019_Total_Population:_40_to_44_Years', '2019_Total_Population:_45_to_49_Years', 
           '2019_Total_Population:_50_to_54_Years', '2019_Total_Population:_55_to_59_Years', 
           '2019_Total_Population:_60_to_64_Years', '2019_Total_Population:_65_to_69_Years',
           '2019_Population_Age_25+_by_Educational_Attainment_Base',
           '2019_Pop_Age_25+:_No_Schooling', '2019_Pop_Age_25+:_9th-12th_(No_Diploma)', 
           '2019_Pop_Age_25+:_High_School_Diploma', '2019_Pop_Age_25+:_GED/Alternative_Credential', 
           '2019_Pop_Age_25+:_Some_College/No_Degree', '2019_Pop_Age_25+:_Associate_Degree',
           '2019_Pop_Age_25+:_Bachelors_Degree', '2019_Pop_Age_25+:_Graduate_Degree',
           '2019_Pop_Age_25+:_Professional_School_Degree', '2019_Pop_Age_25+:_Doctorate_Degree',
           '2019_Median_HH_Income:_HHr_Age_15-24', '2019_Median_HH_Income:_HHr_Age_25-44', 
           '2019_Median_HH_Income:_HHr_Age_45-64', '2019_Median_HH_Income:_HHr_Age_65+', 
           '2021_Median_Household_Income', '2026_Median_Household_Income', 
           '2021_Average_Household_Income', '2026_Average_Household_Income', 
           '2021_2026_Population:_Compound_Annual_Growth_Rate'
           ]

target = ["COSTCO_HEARING_CENTER"]

In [21]:
# Load the data
ml_data_df=ml_data_df.drop(ml_data_df.columns[[0, 2, 3]], axis = 1)

# Drop the null columns where all values are null
ml_datadf = ml_data_df.dropna(axis='columns', how='all')

# Drop the null rows
ml_datadf = ml_data_df.dropna()


ml_data_df.head()

,ZIP,2019_Total_Population:_20_to_24_Years,2019_Total_Population:_25_to_29_Years,2019_Total_Population:_30_to_34_Years,2019_Total_Population:_35_to_39_Years,2019_Total_Population:_40_to_44_Years,2019_Total_Population:_45_to_49_Years,2019_Total_Population:_50_to_54_Years,2019_Total_Population:_55_to_59_Years,2019_Total_Population:_60_to_64_Years,...,2019_Median_HH_Income:_HHr_Age_15-24,2019_Median_HH_Income:_HHr_Age_25-44,2019_Median_HH_Income:_HHr_Age_45-64,2019_Median_HH_Income:_HHr_Age_65+,2021_Median_Household_Income,2026_Median_Household_Income,2021_Average_Household_Income,2026_Average_Household_Income,2021_2026_Population:_Compound_Annual_Growth_Rate,COSTCO_HEARING_CENTER
0,1001,650.0,1027.0,1153.0,773.0,1217.0,1038.0,1331.0,1259.0,1175.0,...,27747.0,78863.0,81899.0,45841.0,68392.0,76174.0,87293.0,98284.0,0.08,None
1,1002,11249.0,2023.0,1344.0,1017.0,951.0,1012.0,1273.0,1241.0,1212.0,...,26373.0,55775.0,98699.0,77235.0,63315.0,71008.0,93933.0,104369.0,0.27,None
2,1003,2734.0,31.0,0.0,7.0,0.0,0.0,0.0,4.0,0.0,...,5000.0,0.0,0.0,87500.0,7500.0,7500.0,12124.0,13505.0,0.00,None
3,1005,414.0,213.0,310.0,121.0,149.0,475.0,368.0,749.0,493.0,...,5000.0,79869.0,84112.0,51184.0,77915.0,88169.0,107888.0,123463.0,0.52,None
4,1007,1252.0,597.0,755.0,987.0,823.0,1113.0,1416.0,969.0,1236.0,...,28089.0,100454.0,112125.0,54294.0,97576.0,104725.0,115051.0,128399.0,0.53,None


In [14]:
#ml_data_df['COSTCO_HEARING_CENTER'] = np.where(ml_data_df['COSTCO_HEARING_CENTER']!='Yes', '0' ,'Yes')

In [22]:
ml_data_df['COSTCO_HEARING_CENTER'] = np.where(ml_data_df['COSTCO_HEARING_CENTER']!='Yes', 0 ,1)

In [28]:
ml_data_df.replace(np.nan, 0, inplace=True)

In [29]:
ml_data_df.COSTCO_HEARING_CENTER.unique()

array([0, 1])

In [30]:
ml_data_df.loc[ml_data_df['COSTCO_HEARING_CENTER']==1]

,ZIP,2019_Total_Population:_20_to_24_Years,2019_Total_Population:_25_to_29_Years,2019_Total_Population:_30_to_34_Years,2019_Total_Population:_35_to_39_Years,2019_Total_Population:_40_to_44_Years,2019_Total_Population:_45_to_49_Years,2019_Total_Population:_50_to_54_Years,2019_Total_Population:_55_to_59_Years,2019_Total_Population:_60_to_64_Years,...,2019_Median_HH_Income:_HHr_Age_15-24,2019_Median_HH_Income:_HHr_Age_25-44,2019_Median_HH_Income:_HHr_Age_45-64,2019_Median_HH_Income:_HHr_Age_65+,2021_Median_Household_Income,2026_Median_Household_Income,2021_Average_Household_Income,2026_Average_Household_Income,2021_2026_Population:_Compound_Annual_Growth_Rate,COSTCO_HEARING_CENTER
53,1089,1771.0,1875.0,1956.0,1816.0,1788.0,2029.0,2492.0,1970.0,1745.0,...,36650.0,58288.0,63810.0,36354.0,56951.0,63351.0,81772.0,93179.0,-0.02,1
275,1923,1498.0,1621.0,1810.0,1649.0,1513.0,1535.0,2160.0,2171.0,1990.0,...,15840.0,107494.0,117775.0,48890.0,94215.0,104306.0,122262.0,139415.0,0.31,1
355,2149,3257.0,4365.0,3670.0,3327.0,3600.0,3097.0,3596.0,2472.0,2246.0,...,55311.0,69087.0,69703.0,39658.0,62887.0,69539.0,79559.0,89554.0,0.64,1
376,2322,160.0,489.0,260.0,396.0,243.0,243.0,442.0,337.0,376.0,...,0.0,125162.0,87772.0,45242.0,81089.0,90724.0,95248.0,105458.0,0.13,1
403,2451,1198.0,1618.0,1869.0,1167.0,871.0,1412.0,1028.0,930.0,955.0,...,91372.0,135967.0,117947.0,68551.0,111013.0,122279.0,140522.0,157154.0,0.19,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
32090,99801,1790.0,2356.0,2204.0,2385.0,1666.0,1876.0,2250.0,2442.0,2035.0,...,75790.0,87277.0,104362.0,71774.0,88010.0,93288.0,108030.0,116121.0,0.37,1
32114,725,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00,1
32115,924,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00,1
32116,959,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00,1


In [31]:
ml_data_df.tail()

,ZIP,2019_Total_Population:_20_to_24_Years,2019_Total_Population:_25_to_29_Years,2019_Total_Population:_30_to_34_Years,2019_Total_Population:_35_to_39_Years,2019_Total_Population:_40_to_44_Years,2019_Total_Population:_45_to_49_Years,2019_Total_Population:_50_to_54_Years,2019_Total_Population:_55_to_59_Years,2019_Total_Population:_60_to_64_Years,...,2019_Median_HH_Income:_HHr_Age_15-24,2019_Median_HH_Income:_HHr_Age_25-44,2019_Median_HH_Income:_HHr_Age_45-64,2019_Median_HH_Income:_HHr_Age_65+,2021_Median_Household_Income,2026_Median_Household_Income,2021_Average_Household_Income,2026_Average_Household_Income,2021_2026_Population:_Compound_Annual_Growth_Rate,COSTCO_HEARING_CENTER
32114,725,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1
32115,924,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1
32116,959,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1
32117,926,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0
32118,961,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1


In [24]:
#ml_data_df = ml_data_df.reset_index()

In [32]:
# Check which columns are non integer/float 
ml_data_df.dtypes[ml_data_df.dtypes != 'int64'][ml_data_df.dtypes != 'float64']

ZIP    int32
dtype: object

# Split the Data into Training and Testing

In [33]:
# Create our features
X = ml_data_df.copy()
X = X.drop(columns="COSTCO_HEARING_CENTER")
X = pd.get_dummies(X)

y = ml_data_df.loc[:, "COSTCO_HEARING_CENTER"]

In [34]:
X.describe()

,ZIP,2019_Total_Population:_20_to_24_Years,2019_Total_Population:_25_to_29_Years,2019_Total_Population:_30_to_34_Years,2019_Total_Population:_35_to_39_Years,2019_Total_Population:_40_to_44_Years,2019_Total_Population:_45_to_49_Years,2019_Total_Population:_50_to_54_Years,2019_Total_Population:_55_to_59_Years,2019_Total_Population:_60_to_64_Years,...,2019_Pop_Age_25+:_Doctorate_Degree,2019_Median_HH_Income:_HHr_Age_15-24,2019_Median_HH_Income:_HHr_Age_25-44,2019_Median_HH_Income:_HHr_Age_45-64,2019_Median_HH_Income:_HHr_Age_65+,2021_Median_Household_Income,2026_Median_Household_Income,2021_Average_Household_Income,2026_Average_Household_Income,2021_2026_Population:_Compound_Annual_Growth_Rate
count,32119.000000,32119.000000,32119.000000,32119.000000,32119.000000,32119.000000,32119.000000,32119.000000,32119.000000,32119.000000,...,32119.000000,32119.000000,32119.000000,32119.000000,32119.000000,32119.000000,32119.00000,32119.000000,32119.000000,32119.000000
mean,50173.424889,686.014291,718.951680,684.539774,656.738441,620.464367,646.075532,665.226533,674.850898,626.544351,...,97.833089,36284.608051,68021.030045,71209.609577,44729.923348,62727.108347,68609.29014,81514.071017,90916.937887,0.321432
std,27411.324266,1238.396776,1218.882419,1132.605342,1040.555320,968.848640,976.324756,962.201754,940.859551,857.982608,...,241.028214,27814.456503,29849.093801,31459.333033,18746.394696,26001.019630,28250.52285,33743.064314,37386.872513,0.871042
min,725.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000,-2.640000
25%,27705.500000,34.000000,35.000000,36.000000,38.000000,39.000000,43.000000,51.000000,57.000000,57.000000,...,1.000000,17946.500000,50468.000000,51988.000000,33944.000000,47402.000000,51644.50000,62014.500000,68896.500000,-0.220000
50%,50164.000000,161.000000,159.000000,159.000000,166.000000,167.000000,186.000000,206.000000,225.000000,216.000000,...,10.000000,33938.000000,63926.000000,66164.000000,41948.000000,57194.000000,62500.00000,73931.000000,82543.000000,0.170000
75%,72349.000000,815.000000,842.000000,835.500000,842.000000,809.000000,864.000000,917.500000,950.000000,891.000000,...,78.000000,50000.000000,80172.000000,83889.000000,52371.000000,72691.500000,79669.00000,91629.000000,102686.000000,0.700000
max,99950.000000,22525.000000,15445.000000,12368.000000,11116.000000,12692.000000,10844.000000,7942.000000,7392.000000,7002.000000,...,4630.000000,200001.000000,200001.000000,200001.000000,200001.000000,200001.000000,200001.00000,346411.000000,372470.000000,23.800000


In [35]:
# Check the balance of our target values
y.value_counts()

0    31565
1      554
Name: COSTCO_HEARING_CENTER, dtype: int64


# Scale the Data

In [36]:
# Create a Standard Scaler instance
data_scaler = StandardScaler()

In [37]:
# Train and transform the data
ml_data_df_encoded_scaled = data_scaler.fit_transform(ml_data_df)

In [38]:
# Preview the scaled data
ml_data_df_encoded_scaled[:5]

array([[-1.79390058, -0.02908184,  0.25273407,  0.41361938,  0.11173204,
         0.61572561,  0.40143463,  0.69193787,  0.62087715,  0.63924861,
         0.66923381,  0.61726709, -0.13857686,  0.28122003,  0.92202602,
         0.14410045,  0.59397444,  0.79476269,  0.48074571,  0.75857818,
        -0.09155632, -0.04909497, -0.30695336,  0.36323176,  0.33978968,
         0.05926974,  0.2178753 ,  0.26777655,  0.17126539,  0.1970525 ,
        -0.27718108, -0.13248051],
       [-1.79386409,  8.52969771,  1.06988878,  0.58225974,  0.34622587,
         0.34116865,  0.37480373,  0.63165851,  0.60174541,  0.6823737 ,
         0.89314358,  0.72770087,  0.30732346, -0.21481806,  0.07156136,
        -0.24306166,  0.06630692,  0.15458287,  0.84722073,  2.20058538,
         1.41626881,  8.47286163, -0.3563529 , -0.4102711 ,  0.87382076,
         1.73396449,  0.02261068,  0.08490983,  0.36804963,  0.3598127 ,
        -0.05904803, -0.13248051],
       [-1.79382761,  1.65376528, -0.56442064, -0.6044


# Split the Data into Training and Testing

In [39]:
# Create X_train, X_test, y_train, y_test
X_train,X_test,y_train,y_test=train_test_split(X,y, random_state=1)

### Balanced Random Forest Classifier

In [40]:
# Resample the training data with the BalancedRandomForestClassifier
rf_model=RandomForestClassifier(n_estimators=100, random_state=43)

In [41]:
# Fitting the model
rf_model = rf_model.fit(X_train, y_train)

In [42]:
# Make the predictions using the test data
predictions=rf_model.predict(X_test)
predictions

array([0, 0, 0, ..., 0, 0, 0])

In [43]:
# Calculated the balanced accuracy score
print("Balanced Random Forest Classifier - Accuracy Score:")
acc_score = balanced_accuracy_score(y_test, predictions)
acc_score

Balanced Random Forest Classifier - Accuracy Score:


0.5084943313028853

In [44]:
# Display the confusion matrix
cm = confusion_matrix(y_test, predictions)

# Create a DataFrame from the confusion matrix.
print("Balanced Random Forest Classifier - Confusion Matrix")

cm_df = pd.DataFrame(
    cm, index=["Actual 0", "Actual 1"], columns=["Predicted 0", "Predicted 1"])

cm_df

Balanced Random Forest Classifier - Confusion Matrix


,Predicted 0,Predicted 1
Actual 0,7912,2
Actual 1,114,2


In [46]:
# Print the imbalanced classification report
print("Balanced Random Forest Classifier - Classification Report")
print(classification_report(y_test, predictions))

Balanced Random Forest Classifier - Classification Report
              precision    recall  f1-score   support

           0       0.99      1.00      0.99      7914
           1       0.50      0.02      0.03       116

    accuracy                           0.99      8030
   macro avg       0.74      0.51      0.51      8030
weighted avg       0.98      0.99      0.98      8030



In [47]:
# Calculate feature importance in the Random Forest model.
importances = rf_model.feature_importances_
importances

array([0.05098605, 0.0317498 , 0.02892596, 0.03412906, 0.03296445,
       0.03302338, 0.03663958, 0.03270807, 0.0307512 , 0.02842697,
       0.02827211, 0.03290798, 0.0322681 , 0.0327323 , 0.02697767,
       0.02788311, 0.03192623, 0.0387045 , 0.05065511, 0.03801203,
       0.03243403, 0.02826482, 0.03180307, 0.02552547, 0.02960341,
       0.03005316, 0.02696693, 0.02786691, 0.02826064, 0.02677733,
       0.03180056])

In [48]:
# List the features sorted in descending order by feature importance
sorted(zip(rf_model.feature_importances_, X.columns), reverse=True)

[(0.05098604744872391, 'ZIP'),
 (0.05065510879322567, '2019_Pop_Age_25+:_Bachelors_Degree'),
 (0.03870450493342664, '2019_Pop_Age_25+:_Associate_Degree'),
 (0.03801203218114847, '2019_Pop_Age_25+:_Graduate_Degree'),
 (0.03663958027757355, '2019_Total_Population:_45_to_49_Years'),
 (0.03412905825927139, '2019_Total_Population:_30_to_34_Years'),
 (0.03302337846848694, '2019_Total_Population:_40_to_44_Years'),
 (0.03296444647038905, '2019_Total_Population:_35_to_39_Years'),
 (0.03290798443274784,
  '2019_Population_Age_25+_by_Educational_Attainment_Base'),
 (0.03273229968695682, '2019_Pop_Age_25+:_9th-12th_(No_Diploma)'),
 (0.03270807087582576, '2019_Total_Population:_50_to_54_Years'),
 (0.03243402837707399, '2019_Pop_Age_25+:_Professional_School_Degree'),
 (0.03226809858505222, '2019_Pop_Age_25+:_No_Schooling'),
 (0.03192623475809363, '2019_Pop_Age_25+:_Some_College/No_Degree'),
 (0.031803073993652176, '2019_Median_HH_Income:_HHr_Age_15-24'),
 (0.031800561083563494, '2021_2026_Population

Instead of having a single, complex tree like the ones created by decision trees, a random forest algorithm will sample the data and build several smaller, simpler decision trees. Each tree is simpler because it is built from a random subset of features.

Random forest algorithms are beneficial because they:
•	Are robust against overfitting as all of those weak learners are trained on different pieces of the data.
•	Can be used to rank the importance of input variables in a natural way.
•	Can handle thousands of input variables without variable deletion.
•	Are robust to outliers and nonlinear data.
•	Run efficiently on large datasets.